# Activity 4: Creating a TensorFlow Model using Keras
In this notebook we design and compile a deep learning model using Keras as an interface to TensorFlow. We will continue to modify this model in our next lessons and activities by experimenting with different optimization techniques. However, the essential components of the model are entirely designed in this notebook.

In [1]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation

D:\Anaconda3\envs\machinelearn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Building a Model

Our dataset contains daily observations and each observation influences a future observation. Also, we are interested in predicting a week--that is, seven days--of Bitcoin prices in the future. For those reasons, we chose the parameters `period_length` and `number_of_observations` as follows:

* `period_length`: the size of the period to use as training input. Our periods are organized in distinct weeks. We will be using a 7-day period to predict a week in the future.
* `number_of_observations`: how many distinct periods does our dataset has? We hvae 77 weeks available in our dataset, given that we will be using the very last week to test the LSTM network on every epoch, we will use 77 - 1 = 76 periods for training it.

In [2]:
period_length = 7
number_of_periods = 76

We now build our LSTM model. 

In [3]:
def build_model(period_length, number_of_periods, batch_size=1):
    """
    Builds an LSTM model using Keras. This function
    works as a simple wrapper for a manually created
    model.
    
    Parameters
    ----------
    period_length: int
        The size of each observation used as input.
    
    number_of_periods: int
        The number of periods available in the 
        dataset.
    
    batch_size: int
        The size of the batch used in each training
        period.
    
    Returns
    -------
    model: Keras model
        Compiled Keras model that can be trained
        and stored in disk.
    """
    model = Sequential()
    model.add(LSTM(
        units=period_length,
        batch_input_shape=(batch_size, number_of_periods, period_length),
        input_shape=(number_of_periods, period_length),
        return_sequences=False, stateful=False))

    model.add(Dense(units=period_length))
    model.add(Activation("linear"))

    model.compile(loss="mse", optimizer="rmsprop")

    return model

### Saving Model

We can use the function `build_model()` as a starting point for building our model. That function will be refactored when building our Flask application for making it easier to train the network and use it for predictions. For now, let's store the model output on disk.

In [4]:
model = build_model(period_length=period_length, number_of_periods=number_of_periods)

In [5]:
model.save('bitcoin_lstm_v0.h5')

The steps above compile the LSTM model as TensorFlow computation graph. We can now train that model using our train and evaluate its results with our test set.